# Milka Vladislav

In [42]:
import pandas as pd

# Завдання: Завантаження даних та обробка DataFrame
# Завантаження Excel-файлу
df = pd.read_excel('En_In.xlsx', sheet_name='Energy', skiprows=17, skipfooter=50, usecols=[2, 3, 5], names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])

# Вивести перші кілька рядків для перевірки
print(df.head())



IndexError: list index out of range